In [ ]:
import graph_fourier_transform
import graph_ruggedness_de
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import networkx as nx
import os
import matplotlib.ticker as ticker
import matplotlib.colors as colors
import matplotlib.cm as cm

### Making KNN graph over GB1 combinatorial dataset 
The below cell is run to make build to KNN graph for the gb1 combinatorial dataset. Note that the `approximate` method is used to find the `k` nearest neighbors due to the time complexity of an exact all vs. all search. 

In [ ]:
df = pd.read_csv('../data_files/gb1_dat_comb.csv')
seq_ls = df['sequences'].tolist()
values = df['fitness'].tolist()
scaler = MinMaxScaler()
values = [val[0] for val in (scaler.fit_transform(np.array(values).reshape(-1,1)))]

G_k = graph_ruggedness_de.build_ohe_graph(seq_ls=seq_ls,
                                        values=values,
                                        edges=False,
                                        hamming_edges=False, 
                                        approximate=True,
                                        n=400)

### Sampling Dirichlet energies over subgraphs 
The below cell is run to sample the Dirichlet energy (approximate method due to comp. complexity) over subgraphs with random sampling proportions defined in `sampling_props` for replicates defined in `replicates`. This indicates how robust quantification of ruggedness via Dirichlet energy is to incomplete graphs. The Dirichlet energy is scaled by the ration between the square-root of the number of nodes in the full graph and the square root of the number of nodes in the sampled graph, as the energy is calculated over the number of edges.

In [ ]:
sampling_props = [0.1, 0.2, 0.5, 0.75, 0.999]
replicates = 10
nodes = G_k.number_of_nodes()
norm_de_dict = {}

for sampling_prop in sampling_props:
    norm_de = []
    gft_coefficients = []
    scaling_factor = np.sqrt(nodes) / np.sqrt(nodes*sampling_prop)
    for _ in range(replicates):

        G_sampled, sampled_nodes, sampled_values = graph_ruggedness_de.sample_graph(G=G_k,
                                                                                    sample_size=sampling_prop)
        graph_ruggedness_de.add_ohe_knn_edges_approx(G=G_sampled,
                                                     k=int(np.sqrt(G_sampled.number_of_nodes())))
        sampled_de = graph_ruggedness_de.compute_dirichlet_energy_approximate(G=G_sampled)
        sampled_de = sampled_de / G_sampled.number_of_nodes()
        sampled_de = sampled_de * scaling_factor
        norm_de.append(sampled_de)
    norm_de_dict[sampling_prop] = np.array(norm_de)

Computing the full KNN graph to determine the actual Dirichlet energy when it is 100% sampled. 

In [ ]:
G_k = graph_ruggedness_de.build_ohe_graph(seq_ls=seq_ls,
                                        values=values,
                                        edges=True,
                                        hamming_edges=False, 
                                        approximate=True,
                                        n=400)
de = graph_ruggedness_de.compute_dirichlet_energy_approximate(G=G_k)

In [ ]:
labs = [key for key in norm_de_dict.keys()]
vals = [norm_de_dict[key] for key in norm_de_dict.keys()]


fig, ax = plt.subplots(figsize=(2.5,2.75))
bp = ax.boxplot(vals, labels=[str(label) for label in labs], notch=False, patch_artist=True)

# Set fill colors for each box
color = 'grey'
for box in bp['boxes']:
    box.set_facecolor(color)

# Set median line color
for median in bp['medians']:
    median.set_color('black')
plt.axhline(de / G_k.number_of_nodes(), color='grey', linestyle='--')

plt.ylim(0,1.5)
plt.ylabel('Normalised Dirichlet energy')
plt.xlabel('Sampling proportion')
plt.xticks(rotation=90)
plt.tight_layout()
plt.savefig('figures/Figure_4/sampling_boxplot.pdf')
plt.show()


### Interpretation of the graph, ruggedness and epistasis over different sampling proportions. 
Different sampling proportions may influence the interpretation of epistasis / ruggedness over the fitness map. Analysing the graph structures, and their local Dirichlet Energies returned through graph sampling is important in understanding how significantly graph sampling can change the biological / biophysical interpretation of these phenomena. For example - does the graph need to be complete to gain an overview of epistasis and ruggedness in that fitness map? 


#### GFT sums

First - this can be tested quantitatively by comparing the cumulative sum the magnitudes from the GFT, when they have been normalised to sum to 1 and the eigenvector indices have been normalised to sum to 1, over both the full GB1 landscape graph and the subsampled GB1 landscape graph (with replication). If the trace of the full GB1 landscape is not significantly different from the distribution of subsampled traces, the information is not meaningfully lost when the graph nodes are subsampled. 

#### Qualitative local Dirichlet Energy analysis
The second approach is a qualitative analysis of the local Dirichlet energy over nodes in both the full and subsampled GFT landscape graphs. If subsampling retains the relative rank of locally rugged nodes, then the biophysical interpretation of epistasis and ruggedness is robust to hamming incomplete graphs. 

In [ ]:
df = pd.read_csv('../data_files/gb1_dat_comb.csv')
seq_ls = df['sequences'].tolist()
values = df['fitness'].tolist()
scaler = MinMaxScaler()
values = [val[0] for val in (scaler.fit_transform(np.array(values).reshape(-1,1)))]

G_k_con = graph_ruggedness_de.build_ohe_graph(seq_ls=seq_ls,
                                        values=values,
                                        edges=True,
                                        hamming_edges=False, 
                                        approximate=True,
                                        n=400)

G_k = graph_ruggedness_de.build_ohe_graph(seq_ls=seq_ls,
                                        values=values,
                                        edges=False,
                                        hamming_edges=False, 
                                        approximate=True,
                                        n=400)

sampling_prop = 0.1
G_sampled, sampled_nodes, sampled_values = graph_ruggedness_de.sample_graph(G=G_k,
                                                                            sample_size=sampling_prop)
graph_ruggedness_de.add_ohe_knn_edges_approx(G=G_sampled,
                                                k=int(np.sqrt(G_sampled.number_of_nodes())))


In [ ]:
graph_ruggedness_de.compute_local_dirichlet_energy(G=G_sampled)

#### Note: the following cell requires > 3 hours to compute. 

In [ ]:
graph_ruggedness_de.compute_local_dirichlet_energy(G=G_k_con,
                                                   approximate=True)

In [ ]:
sampling_prop = 0.1
nodes = G_k.number_of_nodes()

scaling_factor = np.sqrt(nodes) / np.sqrt(nodes*sampling_prop)
G_sampled, sampled_nodes, sampled_values = graph_ruggedness_de.sample_graph(G=G_k,
                                                                            sample_size=sampling_prop)
graph_ruggedness_de.add_ohe_knn_edges_approx(G=G_sampled,
                                            k=int(np.sqrt(G_sampled.number_of_nodes())))

In [ ]:
graph_ruggedness_de.compute_local_dirichlet_energy(G=G_sampled,
                                                   approximate=True)

### Sampling Dirichlet energy over different degrees of incidence
Different degrees of incidence, sampled with `sample_graph_degree`, can be used to gain insight on the contributions of epistasis of n degrees / edges. 

### Epistasis as a function of ruggedness
Starting from a reference node (sequence in the graph), the change in ruggedness as a function of incidence provides insight on the linearity of the fitness function over the system. For example, if the dirichlet energy remains unchanged as the degree increases from a reference node, the fitness of the reference node is not confounded by epistasis. If It linearly increases, the fitness of the reference node is influenced equally by each degree of the graph. If it increases non-linearly, the curvature over the degree can be measured as the sum of discretised second derivatives at each point. If the sum of second derivatives if positive, then non-linearity is injected disproportionately by one degree in the graph. Therefore, the epistasis (or a measurement of it) over a n edge degrees can be described by two values: the gradient in energy over the first through nth degree of the graph (with respect to a particular node) and the curvatrue, or sum of second derivatives. The distribution of these values give insight on the isotropy of the fitness landscape: if there is a wide spread of values, the landscape is anisotropic. 

In [ ]:
df = pd.read_csv('../data_files/gb1_dat_comb.csv')
seq_ls = df['sequences'].tolist()
values = df['fitness'].tolist()
scaler = MinMaxScaler()
values = [val[0] for val in (scaler.fit_transform(np.array(values).reshape(-1,1)))]

G_k = graph_ruggedness_de.build_ohe_graph(seq_ls=seq_ls,
                                        values=values,
                                        edges=True,
                                        hamming_edges=False, 
                                        approximate=True,
                                        n=400)

In [ ]:
G_k_de = graph_ruggedness_de.compute_dirichlet_energy_approximate(G=G_k)

In [ ]:
replicates = 50
degree_dict = {}
for replicate in range(replicates):
    avg_diffs = []
    G_sampled, ref_node, de_degree= graph_ruggedness_de.sample_graph_degree(G=G_k,
                                                        degree=3, 
                                                        compute_de=True,
                                                        approximate=True)
    
    diff_ls = graph_ruggedness_de.count_seq_diff(ref_node=ref_node,
                                                    node_ls=[node for node in G_sampled.nodes()])
    avg_diffs.append(np.mean(diff_ls))
    degree_dict[replicate] = (de_degree, np.array(avg_diffs))

In [ ]:
fig, ax = plt.subplots(figsize=(2.5,2.75))
x_labs = list(range(1,5))
for i in range(50):
    vals = [val for val in degree_dict[i][0].values()]
    vals.append(G_k_de / G_k.number_of_nodes())
    ax.plot(x_labs, vals, color='grey', linestyle='--', linewidth=0.75, alpha=0.35)
plt.xlabel('Distance from ref. (edges)')
plt.ylabel('Normalised Dirichlet energy')
plt.tight_layout()
plt.savefig('figures/Figure_4/de_vs_degree_plot.pdf')
plt.show()


In [ ]:
fig, ax = plt.subplots(figsize=(1.75,1.75))

first_derivative = []
second_derivative = []
for i in range(50):
    vals = [val for val in degree_dict[i][0].values()]
    vals.append(G_k_de / G_k.number_of_nodes())
    derivatives = [vals[i+1] - vals[i] for i in range(len(vals) - 1)]
    first_derivative.append(sum(derivatives))
    second_derivatives = [derivatives[i+1] - derivatives[i] for i in range(len(derivatives) - 1)]
    second_derivative.append(sum(second_derivatives))

plt.hist(first_derivative, bins=10, color='grey', edgecolor='black')
plt.xlabel('Gradient')
plt.ylabel('Count')
plt.tight_layout()
plt.savefig('figures/Figure_4/gradient_hist_full.pdf')
plt.show()

fig, ax = plt.subplots(figsize=(1.75,1.75))
plt.hist(second_derivative, bins=10, color='grey', edgecolor='black')
plt.xlabel('Curvature')
plt.ylabel('Count')
plt.tight_layout()
plt.savefig('figures/Figure_4/curvature_hist_full.pdf')
plt.show()

### Epistasis as a function of ruggedness over incomplete fitness maps 
Measuring epistasis as the gradient and curvature of ruggedness with respect to the degree from a reference node should be (approximately) reproducible between subsampled and complete network Graphs. This can be tested by repeating the previous analyses in subsampled graphs. 

#### Note: the following cells each take > 4 hours to compute. 

In [ ]:
sampling_prop = 0.1
nodes = G_k.number_of_nodes()
sampling_reps = 5
degree_dict_01_ls = []
de_full_01 = {}
for sampling_replicate in range(sampling_reps):

    scaling_factor = np.sqrt(nodes) / np.sqrt(nodes*sampling_prop)
    G_sampled, sampled_nodes, sampled_values = graph_ruggedness_de.sample_graph(G=G_k,
                                                                                sample_size=sampling_prop)
    graph_ruggedness_de.add_ohe_knn_edges_approx(G=G_sampled,
                                                k=int(np.sqrt(G_sampled.number_of_nodes())))
    G_sampled_de = graph_ruggedness_de.compute_dirichlet_energy_approximate(G_sampled) / G_sampled.number_of_nodes()
    de_full_01[sampling_replicate] = G_sampled_de

    replicates = 10
    degree_dict_01 = {}
    for replicate in range(replicates):
        avg_diffs = []
        G_sampled, ref_node, de_degree= graph_ruggedness_de.sample_graph_degree(G=G_sampled,
                                                            degree=3, 
                                                            compute_de=True,
                                                            approximate=True)
        diff_ls = graph_ruggedness_de.count_seq_diff(ref_node=ref_node,
                                                        node_ls=[node for node in G_sampled.nodes()])
        avg_diffs.append(np.mean(diff_ls))
        degree_dict_01[replicate] = (de_degree, np.array(avg_diffs))
        degree_dict_01_ls.append(degree_dict_01)


In [ ]:
fig, ax = plt.subplots(figsize=(1.75,1.75))

first_derivative = []
second_derivative = []
for i in range(5):
    for j in range(10):
        vals = [val for val in degree_dict_01_ls[i][j][0].values()]
        vals.append(de_full_01[i])
        ax.plot(vals, color='grey', linestyle='--', linewidth=0.75, alpha=0.35)
        derivatives = [vals[i+1] - vals[i] for i in range(len(vals) - 1)]
        first_derivative.append(sum(derivatives))
        second_derivatives = [derivatives[i+1] - derivatives[i] for i in range(len(derivatives) - 1)]
        second_derivative.append(sum(second_derivatives))

fig, ax = plt.subplots(figsize=(1.75,1.75))

plt.hist(first_derivative, bins=10, color='grey', edgecolor='black')
plt.xlabel('Gradient')
plt.ylabel('Count')
plt.tight_layout()
plt.show()

fig, ax = plt.subplots(figsize=(1.75,1.75))
plt.hist(second_derivative, bins=10, color='grey', edgecolor='black')
plt.xlabel('Curvature')
plt.ylabel('Count')
plt.tight_layout()
plt.show()

In [ ]:
sampling_prop = 0.5
nodes = G_k.number_of_nodes()
sampling_reps = 5
degree_dict_05_ls = []
de_full_05 = {}
for sampling_replicate in range(sampling_reps):

    scaling_factor = np.sqrt(nodes) / np.sqrt(nodes*sampling_prop)
    G_sampled, sampled_nodes, sampled_values = graph_ruggedness_de.sample_graph(G=G_k,
                                                                                sample_size=sampling_prop)
    G_sampled_de = graph_ruggedness_de.compute_dirichlet_energy_approximate(G_sampled) / G_sampled.number_of_nodes()
    de_full_05[sampling_replicate] = G_sampled_de
    graph_ruggedness_de.add_ohe_knn_edges_approx(G=G_sampled,
                                                k=int(np.sqrt(G_sampled.number_of_nodes())))
    replicates = 10
    degree_dict_05 = {}
    for replicate in range(replicates):
        avg_diffs = []
        G_sampled, ref_node, de_degree= graph_ruggedness_de.sample_graph_degree(G=G_sampled,
                                                            degree=3, 
                                                            compute_de=True,
                                                            approximate=True)
        diff_ls = graph_ruggedness_de.count_seq_diff(ref_node=ref_node,
                                                        node_ls=[node for node in G_sampled.nodes()])
        avg_diffs.append(np.mean(diff_ls))
        degree_dict_05[replicate] = (de_degree, np.array(avg_diffs))
        degree_dict_05_ls.append(degree_dict_05)


In [ ]:
fig, ax = plt.subplots(figsize=(1.75,1.75))

first_derivative = []
second_derivative = []
for i in range(5):
    for j in range(10):
        vals = [val for val in degree_dict_05_ls[i][j][0].values()]
        vals.append(de_full_05[i])
        ax.plot(vals, color='grey', linestyle='--', linewidth=0.75, alpha=0.35)
        derivatives = [vals[i+1] - vals[i] for i in range(len(vals) - 1)]
        first_derivative.append(sum(derivatives))
        second_derivatives = [derivatives[i+1] - derivatives[i] for i in range(len(derivatives) - 1)]
        second_derivative.append(sum(second_derivatives))

fig, ax = plt.subplots(figsize=(1.75,1.75))

plt.hist(first_derivative, bins=10, color='grey', edgecolor='black')
plt.xlabel('Gradient')
plt.ylabel('Count')
plt.tight_layout()
plt.show()

fig, ax = plt.subplots(figsize=(1.75,1.75))
plt.hist(second_derivative, bins=10, color='grey', edgecolor='black')
plt.xlabel('Curvature')
plt.ylabel('Count')
plt.tight_layout()
plt.show()

In [ ]:
vals = []
for i in range(5):
    for j in range(10):
        vals_sample = [val for val in degree_dict_05_ls[i][j][0].values()]

In [ ]:
fig, ax = plt.subplots(figsize=(1.75,1.75))

first_derivative = []
second_derivative = []
for i in range(10):
    for j in range(5):
        vals = [val for val in degree_dict_05_ls[i][j][0].values()]
        ax.plot(vals, color='grey', linestyle='--', linewidth=0.75, alpha=0.35)
        derivatives = [vals[i+1] - vals[i] for i in range(len(vals) - 1)]
        first_derivative.append(sum(derivatives))
        second_derivatives = [derivatives[i+1] - derivatives[i] for i in range(len(derivatives) - 1)]
        second_derivative.append(sum(second_derivatives))

fig, ax = plt.subplots(figsize=(1.75,1.75))

plt.hist(first_derivative, bins=10, color='grey', edgecolor='black')
plt.xlabel('Gradient')
plt.ylabel('Count')
plt.tight_layout()
plt.show()

fig, ax = plt.subplots(figsize=(1.75,1.75))
plt.hist(second_derivative, bins=10, color='grey', edgecolor='black')
plt.xlabel('Curvature')
plt.ylabel('Count')
plt.tight_layout()
plt.show()

In [ ]:
degree_dict_05_ls[i][j-1]

In [ ]:
values = [node[1]['value'] for node in G_sampled.nodes(data=True)]
viridis = plt.cm.get_cmap('viridis', 10)
node_colors = [viridis((value - min(values)) / (max(values) - min(values))) for value in values]
nx.draw(G_sampled, node_color=node_colors, with_labels=False, edgecolors='black', node_size=100, width=0.75, edge_color='grey')
plt.show()

In [ ]:
G_k.number_of_edges()